# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 9:39AM,246916,15,30010865,"Alliance Pharma, Inc.",Released
1,Sep 21 2022 9:39AM,246916,15,30010867,"Alliance Pharma, Inc.",Released
2,Sep 21 2022 9:39AM,246916,15,30010866,"Alliance Pharma, Inc.",Released
3,Sep 21 2022 9:39AM,246916,15,30010868,"Alliance Pharma, Inc.",Released
4,Sep 21 2022 9:39AM,246916,15,30010869,"Alliance Pharma, Inc.",Released
5,Sep 21 2022 9:39AM,246916,15,30010870,"Alliance Pharma, Inc.",Released
6,Sep 21 2022 9:39AM,246916,15,30010871,"Alliance Pharma, Inc.",Released
7,Sep 21 2022 9:39AM,246916,15,30010872,"Alliance Pharma, Inc.",Released
8,Sep 21 2022 9:39AM,246916,15,30010873,"Alliance Pharma, Inc.",Released
9,Sep 21 2022 9:39AM,246916,15,30010875,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
41,246910,Released,11
42,246911,Released,1
43,246914,Released,12
44,246915,Released,1
45,246916,Released,31


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246910,NaN,NaN,11.0
246911,NaN,NaN,1.0
246914,NaN,NaN,12.0
246915,NaN,NaN,1.0
246916,NaN,NaN,31.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,29.0,0.0,3.0
246717,0.0,0.0,1.0
246727,0.0,0.0,1.0
246811,41.0,0.0,8.0
246828,15.0,47.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246711,29,0,3
246717,0,0,1
246727,0,0,1
246811,41,0,8
246828,15,47,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,29,0,3
1,246717,0,0,1
2,246727,0,0,1
3,246811,41,0,8
4,246828,15,47,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246711,29,,3
1,246717,,,1
2,246727,,,1
3,246811,41,,8
4,246828,15,47,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released      int32
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc."
31,Sep 21 2022 9:33AM,246915,19,"GCH Granules USA, Inc."
32,Sep 21 2022 9:31AM,246914,10,Eywa Pharma Inc.
44,Sep 21 2022 9:23AM,246910,20,"Exact-Rx, Inc."
55,Sep 21 2022 9:11AM,246911,19,"GUSA Granules USA, Inc."
56,Sep 21 2022 9:08AM,246904,20,Alumier Labs Inc.
62,Sep 21 2022 8:52AM,246908,16,Sartorius Bioprocess Solutions
67,Sep 21 2022 8:47AM,246907,10,Livs Products
68,Sep 21 2022 8:45AM,246906,10,Livs Products
69,Sep 21 2022 8:33AM,246905,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",,,31
1,Sep 21 2022 9:33AM,246915,19,"GCH Granules USA, Inc.",,,1
2,Sep 21 2022 9:31AM,246914,10,Eywa Pharma Inc.,,,12
3,Sep 21 2022 9:23AM,246910,20,"Exact-Rx, Inc.",,,11
4,Sep 21 2022 9:11AM,246911,19,"GUSA Granules USA, Inc.",,,1
5,Sep 21 2022 9:08AM,246904,20,Alumier Labs Inc.,,,6
6,Sep 21 2022 8:52AM,246908,16,Sartorius Bioprocess Solutions,,,5
7,Sep 21 2022 8:47AM,246907,10,Livs Products,,,1
8,Sep 21 2022 8:45AM,246906,10,Livs Products,,,1
9,Sep 21 2022 8:33AM,246905,10,ISDIN Corporation,,,19


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",31,,
1,Sep 21 2022 9:33AM,246915,19,"GCH Granules USA, Inc.",1,,
2,Sep 21 2022 9:31AM,246914,10,Eywa Pharma Inc.,12,,
3,Sep 21 2022 9:23AM,246910,20,"Exact-Rx, Inc.",11,,
4,Sep 21 2022 9:11AM,246911,19,"GUSA Granules USA, Inc.",1,,
5,Sep 21 2022 9:08AM,246904,20,Alumier Labs Inc.,6,,
6,Sep 21 2022 8:52AM,246908,16,Sartorius Bioprocess Solutions,5,,
7,Sep 21 2022 8:47AM,246907,10,Livs Products,1,,
8,Sep 21 2022 8:45AM,246906,10,Livs Products,1,,
9,Sep 21 2022 8:33AM,246905,10,ISDIN Corporation,19,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",31,,
1,Sep 21 2022 9:33AM,246915,19,"GCH Granules USA, Inc.",1,,
2,Sep 21 2022 9:31AM,246914,10,Eywa Pharma Inc.,12,,
3,Sep 21 2022 9:23AM,246910,20,"Exact-Rx, Inc.",11,,
4,Sep 21 2022 9:11AM,246911,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",31,NaN,NaN
1,Sep 21 2022 9:33AM,246915,19,"GCH Granules USA, Inc.",1,NaN,NaN
2,Sep 21 2022 9:31AM,246914,10,Eywa Pharma Inc.,12,NaN,NaN
3,Sep 21 2022 9:23AM,246910,20,"Exact-Rx, Inc.",11,NaN,NaN
4,Sep 21 2022 9:11AM,246911,19,"GUSA Granules USA, Inc.",1,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 9:39AM,246916,15,"Alliance Pharma, Inc.",31,0.0,0.0
1,Sep 21 2022 9:33AM,246915,19,"GCH Granules USA, Inc.",1,0.0,0.0
2,Sep 21 2022 9:31AM,246914,10,Eywa Pharma Inc.,12,0.0,0.0
3,Sep 21 2022 9:23AM,246910,20,"Exact-Rx, Inc.",11,0.0,0.0
4,Sep 21 2022 9:11AM,246911,19,"GUSA Granules USA, Inc.",1,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3950143,127,15.0,20.0
12,246872,1,0.0,0.0
15,740438,42,0.0,70.0
16,987525,10,9.0,8.0
19,493826,2,0.0,0.0
20,740642,32,47.0,15.0
21,740322,3,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3950143,127,15.0,20.0
1,12,246872,1,0.0,0.0
2,15,740438,42,0.0,70.0
3,16,987525,10,9.0,8.0
4,19,493826,2,0.0,0.0
5,20,740642,32,47.0,15.0
6,21,740322,3,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,127,15.0,20.0
1,12,1,0.0,0.0
2,15,42,0.0,70.0
3,16,10,9.0,8.0
4,19,2,0.0,0.0
5,20,32,47.0,15.0
6,21,3,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,127.0
1,12,Released,1.0
2,15,Released,42.0
3,16,Released,10.0
4,19,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21
Status,,,,,,,
Completed,20.0,0.0,70.0,8.0,0.0,15.0,0.0
Executing,15.0,0.0,0.0,9.0,0.0,47.0,0.0
Released,127.0,1.0,42.0,10.0,2.0,32.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21
0,Completed,20.0,0.0,70.0,8.0,0.0,15.0,0.0
1,Executing,15.0,0.0,0.0,9.0,0.0,47.0,0.0
2,Released,127.0,1.0,42.0,10.0,2.0,32.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21
0,Completed,20.0,0.0,70.0,8.0,0.0,15.0,0.0
1,Executing,15.0,0.0,0.0,9.0,0.0,47.0,0.0
2,Released,127.0,1.0,42.0,10.0,2.0,32.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()